In [46]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from linearmodels.panel import RandomEffects

In [5]:
#df = pd.read_excel("IWA.xlsx", sheet_name='Final Raw Sample(3%)')
df = pd.read_excel("IWA.xlsx", sheet_name='Final Raw Sample(0%)')
df = df.dropna()
df["Revenue"] = df["Total Environmental Cost"]/df["Environmental Intensity (Sales)"]
df["Operating Income"] = df["Total Environmental Cost"]/df["Environmental Intensity (Op Inc)"]
df["Environmental Intensity (Op Inc)"] = df["Environmental Intensity (Op Inc)"]*100
df["Environmental Intensity (Sales)"] = df["Environmental Intensity (Sales)"]*100
print(df.shape)
df.head()

(13255, 36)


,Year,Company Name,Country,GICS Sub-Industry,Industry (Exiobase),Environmental Intensity (Sales),Environmental Intensity (Op Inc),Total Environmental Cost,Working Capacity,Fish Production Capacity,...,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5,% Imputed,Revenue,Operating Income
0,2019,SAGA PLC,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,Multi-line Insurance,Activities auxiliary to financial intermediati...,-2.887178,-13.025357,-3.184231e+07,-3.115075e+07,-7184.203318,...,-4.739468,-1.027193,-3584.970569,-5.649112,70.667599,70.667599,-1297.277948,0.006135,1.102887e+09,244464000.0
1,2019,BURSA MALAYSIA BHD,MALAYSIA,Financial Exchanges & Data,Activities auxiliary to financial intermediati...,-1.677157,-3.465639,-1.968379e+06,-1.924910e+06,-451.342112,...,-1.410813,-1.207108,-222.196310,-1.681590,10.138780,10.138780,-79.398691,0.043215,1.173640e+08,56797000.0
2,2019,INTERTEK GROUP PLC,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,Research & Consulting Services,Activities auxiliary to financial intermediati...,-1.529690,-9.487849,-6.059927e+07,-5.928166e+07,-13774.014902,...,-17.024036,-3.689647,-6861.392776,-20.291452,253.836024,253.836024,-2470.054721,0.011467,3.961539e+09,638704000.0
4,2019,BUREAU VERITAS SA,FRANCE,Research & Consulting Services,Activities auxiliary to financial intermediati...,-0.699273,-5.095678,-3.997865e+07,-3.910761e+07,-9330.459280,...,-37.818819,-9.136488,-4606.916825,-45.077368,586.030400,586.030400,-1632.997165,0.033005,5.717172e+09,784560000.0
5,2019,RICARDO PLC,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,Research & Consulting Services,Activities auxiliary to financial intermediati...,-0.665664,-7.274924,-3.247235e+06,-3.176408e+06,-753.133044,...,-2.096319,-0.454339,-373.412363,-2.498664,31.257053,31.257053,-132.536045,0.025666,4.878190e+08,44636000.0


In [6]:
df_GICSBrew = df[df['GICS Sub-Industry'] == 'Brewers']

# Group the data by "Company Name" and count unique years for each group
company_years_count = df_GICSBrew.groupby("Company Name")["Year"].nunique()

# Filter companies with at least 10 years of data
filtered_companies = company_years_count[company_years_count >= 10].index

# Create a new DataFrame with only the companies that meet the criteria
df_GICSBrewFiltered = df_GICSBrew[df_GICSBrew["Company Name"].isin(filtered_companies)]

print(df_GICSBrewFiltered.shape)
df_GICSBrewFiltered.head()

(70, 36)


,Year,Company Name,Country,GICS Sub-Industry,Industry (Exiobase),Environmental Intensity (Sales),Environmental Intensity (Op Inc),Total Environmental Cost,Working Capacity,Fish Production Capacity,...,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5,% Imputed,Revenue,Operating Income
517,2019,MOLSON COORS BEVERAGE CO,UNITED STATES OF AMERICA,Brewers,Manufacture of beverages,-6.596233,-46.522794,-6.978419e+08,-2.950986e+08,-77656.206578,...,-1113.208360,-326.988191,-37024.568588,-1326.865949,15176.840942,15176.840942,-12261.768244,0.059202,1.057940e+10,1.500000e+09
523,2019,ANHEUSER-BUSCH INBEV,BELGIUM,Brewers,Manufacture of beverages,-3.388736,-11.492977,-1.778566e+09,-1.561186e+09,-391750.191978,...,-3914.291085,-325.789140,-190769.424529,-4665.559245,56659.806086,56659.806086,-64862.799133,0.080199,5.248464e+10,1.547524e+10
526,2019,CARLSBERG A/S,DENMARK,Brewers,Manufacture of beverages,-2.414563,-16.054846,-2.388141e+08,-1.962082e+08,-60262.712495,...,-1480.449400,-288.844325,-27951.868900,-1764.591400,19253.589023,19132.282417,-8148.334758,0.029599,9.890572e+09,1.487489e+09
530,2019,HEINEKEN HOLDING NV,NETHERLANDS,Brewers,Manufacture of beverages,-1.898778,-12.419412,-5.102237e+08,-4.329371e+08,-113003.339951,...,-1458.522182,-168.769751,-54528.158099,-1738.455701,20656.408130,20656.408130,-18050.980779,0.093336,2.687117e+10,4.108276e+09
532,2019,HEINEKEN NV,NETHERLANDS,Brewers,Manufacture of beverages,-1.709847,-10.508374,-4.594560e+08,-4.332461e+08,-113074.003951,...,-1458.522182,-168.769751,-54563.490099,-1738.455701,20656.408130,20656.408130,-18063.841679,0.101854,2.687117e+10,4.372284e+09


In [7]:
df_GICSBrewFiltered_rev = df_GICSBrewFiltered.copy()
for column in df_GICSBrewFiltered_rev.loc[:,"Working Capacity":"SDG 15.5"]:
    df_GICSBrewFiltered_rev[column] = (df_GICSBrewFiltered_rev[column] / df_GICSBrewFiltered_rev['Revenue'])*100
df_GICSBrewFiltered_rev.head()

,Year,Company Name,Country,GICS Sub-Industry,Industry (Exiobase),Environmental Intensity (Sales),Environmental Intensity (Op Inc),Total Environmental Cost,Working Capacity,Fish Production Capacity,...,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5,% Imputed,Revenue,Operating Income
517,2019,MOLSON COORS BEVERAGE CO,UNITED STATES OF AMERICA,Brewers,Manufacture of beverages,-6.596233,-46.522794,-6.978419e+08,-2.789370,-0.000734,...,-0.000011,-3.090801e-06,-0.000350,-0.000013,0.000143,0.000143,-0.000116,0.059202,1.057940e+10,1.500000e+09
523,2019,ANHEUSER-BUSCH INBEV,BELGIUM,Brewers,Manufacture of beverages,-3.388736,-11.492977,-1.778566e+09,-2.974557,-0.000746,...,-0.000007,-6.207324e-07,-0.000363,-0.000009,0.000108,0.000108,-0.000124,0.080199,5.248464e+10,1.547524e+10
526,2019,CARLSBERG A/S,DENMARK,Brewers,Manufacture of beverages,-2.414563,-16.054846,-2.388141e+08,-1.983790,-0.000609,...,-0.000015,-2.920401e-06,-0.000283,-0.000018,0.000195,0.000193,-0.000082,0.029599,9.890572e+09,1.487489e+09
530,2019,HEINEKEN HOLDING NV,NETHERLANDS,Brewers,Manufacture of beverages,-1.898778,-12.419412,-5.102237e+08,-1.611158,-0.000421,...,-0.000005,-6.280700e-07,-0.000203,-0.000006,0.000077,0.000077,-0.000067,0.093336,2.687117e+10,4.108276e+09
532,2019,HEINEKEN NV,NETHERLANDS,Brewers,Manufacture of beverages,-1.709847,-10.508374,-4.594560e+08,-1.612308,-0.000421,...,-0.000005,-6.280700e-07,-0.000203,-0.000006,0.000077,0.000077,-0.000067,0.101854,2.687117e+10,4.372284e+09


In [8]:
df_GICSBrewFiltered_opinc = df_GICSBrewFiltered.copy()
for column in df_GICSBrewFiltered_opinc.loc[:,"Working Capacity":"SDG 15.5"]:
    df_GICSBrewFiltered_opinc[column] = (df_GICSBrewFiltered_opinc[column] / df_GICSBrewFiltered_opinc['Operating Income'])*100
df_GICSBrewFiltered_opinc.head()

,Year,Company Name,Country,GICS Sub-Industry,Industry (Exiobase),Environmental Intensity (Sales),Environmental Intensity (Op Inc),Total Environmental Cost,Working Capacity,Fish Production Capacity,...,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5,% Imputed,Revenue,Operating Income
517,2019,MOLSON COORS BEVERAGE CO,UNITED STATES OF AMERICA,Brewers,Manufacture of beverages,-6.596233,-46.522794,-6.978419e+08,-19.673242,-0.005177,...,-0.000074,-0.000022,-0.002468,-0.000088,0.001012,0.001012,-0.000817,0.059202,1.057940e+10,1.500000e+09
523,2019,ANHEUSER-BUSCH INBEV,BELGIUM,Brewers,Manufacture of beverages,-3.388736,-11.492977,-1.778566e+09,-10.088280,-0.002531,...,-0.000025,-0.000002,-0.001233,-0.000030,0.000366,0.000366,-0.000419,0.080199,5.248464e+10,1.547524e+10
526,2019,CARLSBERG A/S,DENMARK,Brewers,Manufacture of beverages,-2.414563,-16.054846,-2.388141e+08,-13.190567,-0.004051,...,-0.000100,-0.000019,-0.001879,-0.000119,0.001294,0.001286,-0.000548,0.029599,9.890572e+09,1.487489e+09
530,2019,HEINEKEN HOLDING NV,NETHERLANDS,Brewers,Manufacture of beverages,-1.898778,-12.419412,-5.102237e+08,-10.538169,-0.002751,...,-0.000036,-0.000004,-0.001327,-0.000042,0.000503,0.000503,-0.000439,0.093336,2.687117e+10,4.108276e+09
532,2019,HEINEKEN NV,NETHERLANDS,Brewers,Manufacture of beverages,-1.709847,-10.508374,-4.594560e+08,-9.908918,-0.002586,...,-0.000033,-0.000004,-0.001248,-0.000040,0.000472,0.000472,-0.000413,0.101854,2.687117e+10,4.372284e+09


## Helper Functions

In [45]:
harvard_set = ["Working Capacity", "Fish Production Capacity", "Crop Production Capacity",
                      "Meat Production Capacity", "Biodiversity", "Abiotic Resources",
                      "Water production capacity (Drinking water & Irrigation Water)", "Wood Production Capacity"]

sdg_set = ["SDG 1.5", "SDG 2.1", "SDG 2.2", "SDG 2.3", "SDG 2.4", "SDG 3.3", 
                         "SDG 3.4", "SDG 3.9", "SDG 6", "SDG 12.2", "SDG 14.1", "SDG 14.2", 
                         "SDG 14.3", "SDG 14.c", "SDG 15.1", "SDG 15.2", "SDG 15.5"]

def getXYVal(isHarvard, isRevenue, isAbsolute = False):
    """
        Sets up the dependent and independent variables to be used by the regressions

    Args:
        isHarvard (bool): check if using the harvard targets or SDG targets
        isRevenue (bool): check if using the revenue or operating income
        isAbsolute (bool, optional): Optional to check if using the absolute values or percentage. Defaults to False.

    Returns:
        None
    """
    selectedSet = harvard_set if isHarvard else sdg_set   
    dependent_variable  = "Revenue" if isRevenue else "Operating Income"
    
    df_source = df_GICSBrewFiltered_rev if isRevenue else df_GICSBrewFiltered_opinc
    df_source = df_GICSBrewFiltered if isAbsolute else df_source
        
    X = df_source[selectedSet]
    X = sm.add_constant(X)
    
    y = df_source[dependent_variable]
    
    return X, y

In [26]:
def runLasso(isHarvard, isRevenue, isAbsolute = False):
    """
        Runs the lasso regression with the settings passed as parameters.

    Args:
        isHarvard (bool): check if using the harvard targets or SDG targets
        isRevenue (bool): check if using the revenue or operating income
        isAbsolute (bool, optional): Optional to check if using the absolute values or percentage. Defaults to False.

    Returns:
        None
    """
    
    selectedSet = harvard_set if isHarvard else sdg_set       
    X,y = getXYVal(isHarvard=isHarvard, isRevenue=isRevenue, isAbsolute=isAbsolute)
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Create and fit the Lasso regression model
    alpha = 1.0  # You can adjust the regularization strength (alpha) as needed
    lasso_model = Lasso(alpha=alpha)
    lasso_model.fit(X_train, y_train)
    
    # Get the coefficients (weights) of the independent variables
    coefficients = lasso_model.coef_

    # Print the coefficients
    print("Lasso Coefficients:")
    for i in range(len(coefficients)):
        if i == 0: continue
        print(f"{selectedSet[i-1]}: {coefficients[i]:.4f}")

In [77]:
def runRandomEffects(isHarvard, isRevenue, isAbsolute = False):
    """
        Runs the Randome Effects model with the settings passed as parameters.

    Args:
        isHarvard (bool): check if using the harvard targets or SDG targets
        isRevenue (bool): check if using the revenue or operating income
        isAbsolute (bool, optional): Optional to check if using the absolute values or percentage. Defaults to False.

    Returns:
        None
    """
    
    entity_column = 'Company Name'
    time_column = 'Year'
    
    independent_variables = harvard_set if isHarvard else sdg_set   
    dependent_variable  = "Revenue" if isRevenue else "Operating Income"

    df_source = df_GICSBrewFiltered_rev if isRevenue else df_GICSBrewFiltered_opinc
    df_source = df_GICSBrewFiltered if isAbsolute else df_source
    
    # Create a panel data structure with its entity and time identifiers
    data = df_source.set_index([entity_column, time_column])
    
    # Create a random effects model
    model = RandomEffects(data[dependent_variable], data[independent_variables], check_rank=False)

    # Fit the model
    results = model.fit()

    # Print the summary of the model
    print(results)

## Run Lasso Regression
### Revenue using Harvard targets

In [27]:
runLasso(isHarvard=True, isRevenue=True)

Lasso Coefficients:
Working Capacity: -18188837969.8223
Fish Production Capacity: -5587025890165.8096
Crop Production Capacity: -2287988405083.2036
Meat Production Capacity: 13793380420463.7500
Biodiversity: 52185343852533.2109
Abiotic Resources: 5740391484601.8682
Water production capacity (Drinking water & Irrigation Water): 11754632885.8448
Wood Production Capacity: 27988918887937.5781


c:\Users\ianpe\mambaforge\envs\PythonBase\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.014e+21, tolerance: 7.041e+17
  model = cd_fast.enet_coordinate_descent(


### Revenue using SDG targets

In [28]:
runLasso(isHarvard=False, isRevenue=True)

Lasso Coefficients:
SDG 1.5: 14187122380.8971
SDG 2.1: -74017981264.8570
SDG 2.2: 5362887336.1116
SDG 2.3: -440715040330.7382
SDG 2.4: -22249526974.6296
SDG 3.3: -15941115300.5793
SDG 3.4: -400317103136.8707
SDG 3.9: 18950335496203.8711
SDG 6: 1382621651.7896
SDG 12.2: -832407141947.4319
SDG 14.1: -7857405418913675.0000
SDG 14.2: 16122365257433124.0000
SDG 14.3: -317941069475066.2500
SDG 14.c: 1057412384136798.1250
SDG 15.1: 19956242853738.0586
SDG 15.2: -7637356221552.6650
SDG 15.5: 1191688032303269.2500


c:\Users\ianpe\mambaforge\envs\PythonBase\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.546e+20, tolerance: 7.041e+17
  model = cd_fast.enet_coordinate_descent(


### Op Inc using Harvard targets

In [31]:
runLasso(isHarvard=True, isRevenue=False)

Lasso Coefficients:
Working Capacity: -119962810.0069
Fish Production Capacity: 1569697212406.0112
Crop Production Capacity: -294164027877.0670
Meat Production Capacity: 1323441454620.3181
Biodiversity: -3487563277885.9731
Abiotic Resources: 162924940495.7467
Water production capacity (Drinking water & Irrigation Water): 329043369.2720
Wood Production Capacity: 1158904140776.1570


c:\Users\ianpe\mambaforge\envs\PythonBase\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.470e+20, tolerance: 8.290e+16
  model = cd_fast.enet_coordinate_descent(


### Op Inc using SDG targets

In [49]:
runLasso(isHarvard=False, isRevenue=False)

Lasso Coefficients:
SDG 1.5: 2729906554.5849
SDG 2.1: -2110095791.5534
SDG 2.2: 599936712.3495
SDG 2.3: -545009627652.3052
SDG 2.4: -6283068680.8203
SDG 3.3: 39282622979.1285
SDG 3.4: -36858519302.3265
SDG 3.9: -26678743542629.5781
SDG 6: -181378087.5411
SDG 12.2: 2401287273223.5894
SDG 14.1: -396982663056159.8750
SDG 14.2: 1723576253225897.5000
SDG 14.3: -34185255992257.1719
SDG 14.c: -83572106249786.2188
SDG 15.1: 16050144303885.7109
SDG 15.2: -1391221442582.6824
SDG 15.5: 84075865028778.5938


c:\Users\ianpe\mambaforge\envs\PythonBase\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.172e+20, tolerance: 8.290e+16
  model = cd_fast.enet_coordinate_descent(


## Run Random Effects Model
### Revenue using Harvard targets

In [56]:
runRandomEffects(isHarvard=True, isRevenue=True)

                        RandomEffects Estimation Summary                        
Dep. Variable:                Revenue   R-squared:                        0.9126
Estimator:              RandomEffects   R-squared (Between):              0.9816
No. Observations:                  70   R-squared (Within):              -1.2912
Date:                Fri, Oct 06 2023   R-squared (Overall):              0.9126
Time:                        01:38:49   Log-likelihood                   -1683.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      80.971
Entities:                           7   P-value                           0.0000
Avg Obs:                      10.0000   Distribution:                    F(8,62)
Min Obs:                      10.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             80.971
                            

### Revenue using SDG targets

In [78]:
runRandomEffects(isHarvard=False, isRevenue=True)

                        RandomEffects Estimation Summary                        
Dep. Variable:                Revenue   R-squared:                        0.7776
Estimator:              RandomEffects   R-squared (Between):              0.9883
No. Observations:                  70   R-squared (Within):              -0.4998
Date:                Fri, Oct 06 2023   R-squared (Overall):              0.7776
Time:                        01:50:15   Log-likelihood                   -1662.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      11.580
Entities:                           7   P-value                           0.0000
Avg Obs:                      10.0000   Distribution:                   F(16,53)
Min Obs:                      10.0000                                           
Max Obs:                      10.0000   F-statistic (robust):            -35.663
                            

### Op Inc using Harvard targets

In [79]:
runRandomEffects(isHarvard=True, isRevenue=False)

                        RandomEffects Estimation Summary                        
Dep. Variable:       Operating Income   R-squared:                        0.8613
Estimator:              RandomEffects   R-squared (Between):              0.9617
No. Observations:                  70   R-squared (Within):              -1.3367
Date:                Fri, Oct 06 2023   R-squared (Overall):              0.8613
Time:                        01:52:39   Log-likelihood                   -1592.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      48.126
Entities:                           7   P-value                           0.0000
Avg Obs:                      10.0000   Distribution:                    F(8,62)
Min Obs:                      10.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             48.126
                            

### Op Inc using SDG targets

In [81]:
runRandomEffects(isHarvard=False, isRevenue=False)

                        RandomEffects Estimation Summary                        
Dep. Variable:       Operating Income   R-squared:                        0.9100
Estimator:              RandomEffects   R-squared (Between):              0.9993
No. Observations:                  70   R-squared (Within):              -0.0978
Date:                Fri, Oct 06 2023   R-squared (Overall):              0.9100
Time:                        01:53:12   Log-likelihood                   -1555.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      33.509
Entities:                           7   P-value                           0.0000
Avg Obs:                      10.0000   Distribution:                   F(16,53)
Min Obs:                      10.0000                                           
Max Obs:                      10.0000   F-statistic (robust):            -368.29
                            

c:\Users\ianpe\mambaforge\envs\PythonBase\Lib\site-packages\linearmodels\panel\results.py:88: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")


## FIXED EFFECTS

In [124]:
import pandas as pd
from linearmodels.panel import PanelOLS



Company Name  \
Industry (Exiobase)                                Year                             
Activities auxiliary to financial intermediatio... 2019-01-01            SAGA PLC   
                                                   2019-01-01  BURSA MALAYSIA BHD   
                                                   2019-01-01  INTERTEK GROUP PLC   
                                                   2019-01-01         JSE LIMITED   
                                                   2019-01-01   BUREAU VERITAS SA   

                                                                                                         Country  \
Industry (Exiobase)                                Year                                                            
Activities auxiliary to financial intermediatio... 2019-01-01  UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...   
                                                   2019-01-01                                           MALAYSIA   
                                                   2019-01-01  UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...   
                                                   2019-01-01                                       SOUTH AFRICA   
                                                   2019-01-01                                             FRANCE   

                                                                            GICS Sub-Industry  \
Industry (Exiobase)                                Year                                         
Activities auxiliary to financial intermediatio... 2019-01-01            Multi-line Insurance   
                                                   2019-01-01      Financial Exchanges & Data   
                                                   2019-01-01  Research & Consulting Services   
                                                   2019-01-01      Financial Exchanges & Data   
                                                   2019-01-01  Research & Consulting Services   

                                                               Environmental Intensity (Sales)  \
Industry (Exiobase)                                Year                                          
Activities auxiliary to financial intermediatio... 2019-01-01                        -2.887178   
                                                   2019-01-01                        -1.677157   
                                                   2019-01-01                        -1.529690   
                                                   2019-01-01                        -1.462497   
                                                   2019-01-01                        -0.699273   

                                                               Environmental Intensity (Op Inc)  \
Industry (Exiobase)                                Year                                           
Activities auxiliary to financial intermediatio... 2019-01-01                        -13.025357   
                                                   2019-01-01                         -3.465639   
                                                   2019-01-01                         -9.487849   
                                                   2019-01-01                               NaN   
                                                   2019-01-01                         -5.095678   

                                                               Total Environmental Cost  \
Industry (Exiobase)                                Year                                   
Activities auxiliary to financial intermediatio... 2019-01-01             -3.184231e+07   
                                                   2019-01-01             -1.968379e+06   
                                                   2019-01-01             -6.059927e+07   
                                                   2019-01-01             -2.290124e+06   
                                                   20

In [129]:
fixedEffects_df = df.copy()
fixedEffects_df['Year'] = pd.to_datetime(fixedEffects_df['Year'], format='%Y')

# Set 'Year' as the time variable and 'Company Name' as the entity variable
fixedEffects_df.set_index(['Industry (Exiobase)', 'Year'], inplace=True)

y = fixedEffects_df['Environmental Intensity (Sales)']
X = fixedEffects_df[["Working Capacity", "Fish Production Capacity", "Crop Production Capacity",
                      "Meat Production Capacity", "Biodiversity", "Abiotic Resources",
                      "Water production capacity (Drinking water & Irrigation Water)", "Wood Production Capacity"]]

fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True, drop_absorbed=True, check_rank=False)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary

Dep. Variable:,Environmental Intensity (Sales),R-squared:,0.1826
Estimator:,PanelOLS,R-squared (Between):,0.2766
No. Observations:,14035,R-squared (Within):,0.1823
Date:,"Mon, Oct 02 2023",R-squared (Overall):,0.2603
Time:,16:00:24,Log-likelihood,-6.088e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,389.99
Entities:,56,P-value,0.0000
Avg Obs:,250.62,Distribution:,"F(8,13962)"
Min Obs:,1.0000,,
Max Obs:,1463.0,F-statistic (robust):,389.99


In [130]:
fixedEffects_df = df.copy()
fixedEffects_df['Year'] = pd.to_datetime(fixedEffects_df['Year'], format='%Y')

# Set 'Year' as the time variable and 'Company Name' as the entity variable
fixedEffects_df.set_index(['GICS Sub-Industry', 'Year'], inplace=True)
#fixedEffects_df["Revenue"] = fixedEffects_df["Total Environmental Cost"]/fixedEffects_df["Environmental Intensity (Sales)"]
fixedEffects_df.head()

# Define the dependent variable (Y) and independent variables (X1-X15)
#y = np.log(fixedEffects_df['Revenue'])
y = fixedEffects_df['Environmental Intensity (Sales)']
X = fixedEffects_df[["Working Capacity", "Fish Production Capacity", "Crop Production Capacity",
                      "Meat Production Capacity", "Biodiversity", "Abiotic Resources",
                      "Water production capacity (Drinking water & Irrigation Water)", "Wood Production Capacity"]]

fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True, drop_absorbed=True, check_rank=False)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary

Dep. Variable:,Environmental Intensity (Sales),R-squared:,0.1733
Estimator:,PanelOLS,R-squared (Between):,0.2989
No. Observations:,14035,R-squared (Within):,0.1730
Date:,"Mon, Oct 02 2023",R-squared (Overall):,0.2587
Time:,16:00:31,Log-likelihood,-6.03e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,363.28
Entities:,153,P-value,0.0000
Avg Obs:,91.732,Distribution:,"F(8,13865)"
Min Obs:,1.0000,,
Max Obs:,521.00,F-statistic (robust):,363.28


In [143]:
X = df[["Year"]]
y = np.log(df[["Environmental Intensity (Sales)"]])
X = sm.add_constant(X)

# Fit the random-effects panel regression model
model = sm.OLS(y, X).fit()

# Get the regression results
results = model.summary()
print(results)

                                   OLS Regression Results                                  
Dep. Variable:     Environmental Intensity (Sales)   R-squared:                         nan
Model:                                         OLS   Adj. R-squared:                    nan
Method:                              Least Squares   F-statistic:                       nan
Date:                             Mon, 02 Oct 2023   Prob (F-statistic):                nan
Time:                                     16:17:48   Log-Likelihood:                    nan
No. Observations:                            14035   AIC:                               nan
Df Residuals:                                14033   BIC:                               nan
Df Model:                                        1                                         
Covariance Type:                         nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-

c:\Users\ianpe\mambaforge\envs\PythonBase\Lib\site-packages\pandas\core\internals\blocks.py:366: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
